In [2]:
from nltk.tokenize import sent_tokenize, word_tokenize



In [54]:
import requests

link = "https://www.4icu.org/us/"
f = requests.get(link)
# print(f.text)

# print(f.text)

import re

university_list = re.findall("<td><a href=\S+>(\D+)</a>", f.text)
for x in range(len(university_list)):
    curr = university_list[x]
    if curr.lower() == "columbia university in the city of new york":
        university_list[x] = curr[:19] 
    if "&amp" in curr:
        university_list[x].replace("&amp;", " ")
        # and_index = curr.index("&")
        # university_list[x] = curr[0:and_index] + " " + curr[and_index+5:]
    if "The" in curr:
        university_list[x] = curr[3:]
    if "," in curr:
        university_list[x].replace(",", "")

print(len(university_list))

top_400 = university_list[0:400]

# print(top_200)


1766


In [ ]:
import requests

link = "https://www.collegevine.com/schools/hub/overall/sheet"
f = requests.get(link)
# print(f.text)

# <h4 class="text-primary mb-0 cursor-pointer">Stanford University</h4>

import re

university_list = re.findall("<h4 class=\"text-primary mb-0 cursor-pointer\">(.+)</h4>", f.text)

# for x in range(len(university_list)):
#     curr = university_list[x]

print(len(university_list))

top_400 = university_list[0:400]

# print(top_200)


1766


In [11]:
# get links for all schools

college_links = []

for name in top_400:
    name = name.lower()
    college_links.append("https://www.collegevine.com/schools/" 
                            + name.replace(" ", "-") 
                            +  "/essay-prompts")

print(college_links[0])

https://www.collegevine.com/schools/massachusetts-institute-of-technology/essay-prompts


In [43]:
# access links

potential_issues = []

for link in college_links:
    temp_f = requests.get(link)
    prompts = re.findall("<p>(.+)</p>", temp_f.text)

    if prompts == []:
        potential_issues.append(link)


# print(college_links[333])

# temp_f = requests.get(college_links[323])

# prompts = re.findall("<p>(.+)</p>", temp_f.text)
# print(prompts)


In [45]:
len(potential_issues)

print(potential_issues)

['https://www.collegevine.com/schools/university-of-california,-berkeley/essay-prompts', 'https://www.collegevine.com/schools/columbia-university-in-the-city-of-new-york/essay-prompts', 'https://www.collegevine.com/schools/university-of-california,-los-angeles/essay-prompts', 'https://www.collegevine.com/schools/the-university-of-texas-at-austin/essay-prompts', 'https://www.collegevine.com/schools/penn-state-university/essay-prompts', 'https://www.collegevine.com/schools/michigan-state-university/essay-prompts', 'https://www.collegevine.com/schools/university-of-california,-san-diego/essay-prompts', 'https://www.collegevine.com/schools/university-of-california,-davis/essay-prompts', 'https://www.collegevine.com/schools/university-of-maryland/essay-prompts', 'https://www.collegevine.com/schools/rutgers,-the-state-university-of-new-jersey/essay-prompts', 'https://www.collegevine.com/schools/university-of-california,-irvine/essay-prompts', 'https://www.collegevine.com/schools/the-universi

In [52]:
for x in top_400:
    if " in " in x.lower():
        print(x)

Columbia University in the City of New York
Washington University in St. Louis
The University of Alabama in Huntsville


In [ ]:
# do this
# conda env export > environment.yml